# Agrupación y Segmentación de Vecindarios en Toronto
## Parte 2

Explore y agrupe los vecindarios de Toronto. Puede decidir trabajar sólo con los municipios que contienen la palabra Toronto y luego replicar el mismo análisis que hicimos con los datos de la ciudad de Nueva York. Es su decisión.

Sólo asegúrese:

1.     Agregar suficientes celdas de Anotación para explicar lo que decidió hacer y para informar cualquier observación que haga.


Primero, importamos las librerías necesarias.
Después leemos los resultados del ejercicio anterior.

In [1]:

# Importamos "Numpy" como 'np'
import numpy as np
# Importamos "Pandas" como 'pd'
import pandas as pd
# Importamos 'fsqApiKey' desde "config"
from config import fsqApiKey
# Importamos "Requests"
import requests
# Leemos el archivo del ejercicio anterior y lo asignamos a la variable "df_merged"
df_merged = pd.read_csv("Toronto_Q2.csv")

### Funciones predefinidas en Labs anteriores

In [25]:
# Función "getNearbyVenues"
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 50
    
    headers = {
        'Accept': 'application/json',
        'Authorization': fsqApiKey
    }

    URL = 'https://api.foursquare.com/v3/places/search?ll={},{}&radius={}&limit={}'

    venues_list = []
    

    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        url = URL.format(lat, lng, radius, LIMIT)
        data = requests.get(url, headers=headers).json()

    # Procesamos venues_list
    for record in data['results']:
        if not record['categories']:
            # La lista está vacía, ignoramos el registro actual
            print(record['name'] + " no tiene categoría")
            continue
        else:
            venues_list.append({
                'Vecindario': name,
                'Latitud': lat,
                'Longitud': lng,
                'Venue': record['name'],
                'Venue Latitude': record['geocodes']['main']['latitude'],
                'Venue Longitude': record['geocodes']['main']['longitude'],
                'Venue Category': record['categories'][0]['name'],
    })

    # Creamos el dataFrame 'nearby_venues' con 'venues_list'
    column_names = ['Vecindario', 
                  'Latitud', 
                  'Longitud', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    nearby_venues = pd.DataFrame(columns=column_names)

    return(nearby_venues)


# Función que nos devuelve la categoría de cada sitio
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

# Función quenos devuelve los lugares con más puntuación
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Importamos el resto de librerías que nos faltan:

In [22]:
# Importamos la librería JSON
import json 

# Instalamos geopy (En este caso ya está instalado, en caso contrario deshaz el comentario)
# c!onda install -c conda-forge geopy --yes 

# Importamos "Nominatim" desde "geopy.geocoders" para convertir una direción en coordenadas
from geopy.geocoders import Nominatim

# Importamos "Matplotlib.cm" como 'cm'
import matplotlib.cm as cm
# Importamos "Matplotlib.colors" como 'colors'
import matplotlib.colors as colors

# Importamos "KMeans" desde "sklearn.cluster"
from sklearn.cluster import KMeans

# Importamos "Folium" (En este caso ya está instalado, en caso contrario deshaz el comentario)
# c!onda install -c conda-forge folium=0.5.0 --yes

#Importamos "Folium"
import folium

print('✅  Todas las librerías se han importado correctamente ✅')

✅  Todas las librerías se han importado correctamente ✅


2.     Generar mapas para visualizar sus vecindarios y cómo se agrupan.

Dibujamos un mapa de Toronto con todos los Vecindarios:

In [23]:

#43.628841 	-79.520999
latitude = 43.628841
longitude = -79.520999


# create map of New York using latitude and longitude values
mapa_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, municipio, vecindario in zip(df_merged['Latitud'], df_merged['Longitud'], df_merged['Municipio'], df_merged['Vecindario']):
    label = '{}, {}'.format(vecindario, municipio)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(mapa_toronto)  
    
mapa_toronto



Cogemos sólo los datos de las filas que contengan "Toronto"

In [26]:
datos = df_merged.set_index('Municipio').filter(regex=".*Toronto",axis=0)
datos.head()



,CódigoPostal,Vecindario,Latitud,Longitud
Municipio,,,,
Downtown Toronto,M5A,Regent Park / Harbourfront,43.654260,-79.360636
Downtown Toronto,M5B,"Garden District, Ryerson",43.657162,-79.378937
Downtown Toronto,M5C,St. James Town,43.651494,-79.375418
East Toronto,M4E,The Beaches,43.676357,-79.293031
Downtown Toronto,M5E,Berczy Park,43.644771,-79.373306


Usamos "getNearbyVenues" con los datos de Toronto.

In [27]:



venuesToronto = getNearbyVenues(names = datos['Vecindario'],
                                   latitudes = datos['Latitud'],
                                   longitudes = datos['Longitud'])

Regent Park / Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond / Adelaide / King
Dufferin / Dovercourt Village
The Danforth East
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
The Danforth West / Riverdale
Toronto Dominion Centre / Design Exchange
Brockton / Parkdale Village / Exhibition Place
India Bazaar / The Beaches West
Commerce Court / Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park / The Junction South
North Toronto West
The Annex / North Midtown / Yorkville
Parkdale / Roncesvalles
Davisville
University of Toronto / Harbord
Runnymede / Swansea
Moore Park / Summerhill East
Kensington Market / Chinatown / Grange Park
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Rosedale
Enclave of M5E
St. James

In [9]:
print('El tamaño del dataFrame es de {}'.format(venuesToronto.shape))
venuesToronto.head()
venuesToronto.groupby('Vecindario').count()
print('Hay {} categorías únicas'.format(len(venuesToronto['Venue Category'].unique())))

El tamaño del dataFrame es de (45, 7)
Hay 40 categorías únicas


Hacemos One Hot Encoding

In [19]:
# Codificación One Hot
toronto_onehot = pd.get_dummies(venuesToronto[['Venue Category']], prefix="", prefix_sep="")

# Añadimos la columna 'Vecindario' de vuelta al dataframe
toronto_onehot['Vecindario'] = venuesToronto['Vecindario'] 

# Movemos la columna 'Vecindario' a la primera columna
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Vecindario,Antique Store,Audiovisual Service,Automotive Repair Shop,Automotive Service,Bathroom Contractor,Bicycle Store,Boutique,Brewery,Burger Joint,...,Photographer,Pizzeria,Playground,Public and Social Service,Recording Studio,Sandwich Restaurant,Sports and Recreation,Storage Facility,Theater,Yoga Studio
0,Enclave of M4L,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Enclave of M4L,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Enclave of M4L,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Enclave of M4L,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Enclave of M4L,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Agrupamos las filas por barrios tomando la media de la frecuencia de cada categoría

In [9]:
toronto_agrupado = toronto_onehot.groupby('Vecindario').mean().reset_index()
toronto_agrupado

,Vecindario,Antique Store,Audiovisual Service,Automotive Repair Shop,Automotive Service,Bathroom Contractor,Bicycle Store,Boutique,Brewery,Burger Joint,...,Photographer,Pizzeria,Playground,Public and Social Service,Recording Studio,Sandwich Restaurant,Sports and Recreation,Storage Facility,Theater,Yoga Studio
0,Enclave of M4L,0.022222,0.022222,0.044444,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,...,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.044444,0.022222,0.022222


Examinamos el tamaño del nuevo dataframe

In [10]:
toronto_agrupado.shape

(1, 41)

#### Imprimimos cada barrio junto con los 5 sitios más comunes

In [14]:
num_top_venues = 5

for hood in toronto_agrupado['Vecindario']:
    print("----"+hood+"----")
    temp = toronto_agrupado[toronto_agrupado['Vecindario'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Enclave of M4L----
                                venue  freq
0              Automotive Repair Shop  0.04
1           Health and Beauty Service  0.04
2                    Storage Facility  0.04
3             Landscaper and Gardener  0.04
4  Business and Professional Services  0.04




Una vez que esté satisfecho con su análisis, envíe un enlace al nuevo Notebook en su repositorio de Github. (3 puntos)